In [1]:
%load_ext rich

### Cross check environment

In [2]:
import logging
import os

print(os.getenv("CONDA_BUILD_SYSROOT"))
print(os.getenv("ROOUNFOLD_ROOT"))
print(os.getenv("LD_LIBRARY_PATH"))
print(os.getenv("PATH"))

None
None
None
/Users/REhlers/software/dev/mammoth/.venv/bin:/Users/REhlers/.pyenv/shims:/opt/homebrew/Caskroom/mambaforge/base/condabin:/opt/homebrew/opt/node@16/bin:/Users/REhlers/Library/Python/3.10/bin:/Users/REhlers/install/bin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/TeX/texbin:/opt/X11/bin:/Users/REhlers/.local/bin


# Running jobs

## General setup

In [3]:
from importlib import reload
from pathlib import Path
from typing import List

from mammoth import job_utils

from mammoth.hardest_kt import produce_flat_skim_from_track_skim

## Reload

In [ ]:
#reload(job_utils)
reload(produce_flat_skim_from_track_skim)

## Tasks

### Running

In [4]:
def define_productions() -> List[produce_flat_skim_from_track_skim.production.ProductionSettings]:
    # We want to provide the opportunity to run multiple productions at once.
    # We'll do so by defining each production below and then iterating over them below
    productions = []

    # Create and store production information
    _here = Path(produce_flat_skim_from_track_skim.__file__).parent
    config_filename = Path(_here.parent / "alice" / "config" / "track_skim_config.yaml")
    productions.extend(
        [
            # Debug
            # production.ProductionSettings.read_config(
            #     collision_system="embed_pythia", number=3,
            #     specialization=HardestKtProductionSpecialization(),
            #     track_skim_config_filename=config_filename,
            # ),
            # Production
            # produce_flat_skim_from_track_skim.production.ProductionSettings.read_config(
            #     collision_system="pp",
            #     number=60,
            #     specialization=produce_flat_skim_from_track_skim.HardestKtProductionSpecialization(),
            #     track_skim_config_filename=config_filename,
            # ),
            produce_flat_skim_from_track_skim.production.ProductionSettings.read_config(
                collision_system="pythia",
                number=60,
                specialization=produce_flat_skim_from_track_skim.HardestKtProductionSpecialization(),
                track_skim_config_filename=config_filename,
            ),
        ]
    )

    # Write out the production settings
    for production_settings in productions:
        production_settings.store_production_parameters()

    return productions


In [5]:
# Job execution parameters
productions = define_productions()

# Base settings
job_framework = job_utils.JobFramework.dask_delayed
#job_framework = job_utils.JobFramework.immediate_execution_debug
facility: job_utils.FACILITIES = "rehlers_mbp_m1pro"
#conda_environment_name = "substructure_c_24_06"
conda_environment_name = ""

# Job execution configuration
task_name = "hardest_kt_mammoth"
task_config = job_utils.TaskConfig(name=task_name, n_cores_per_task=1)
if task_config.n_cores_per_task > 1:
    facility = "rehlers_mbp_m1pro_multi_core"
# Formerly n_cores_to_allocate, this new variable is == n_cores_to_allocate if n_cores_per_task == 1
target_n_tasks_to_run_simultaneously = 8
log_level = logging.INFO
walltime = "24:00:00"
debug_mode = False

if debug_mode:
    # Usually, we want to run in the short queue
    target_n_tasks_to_run_simultaneously = 4
    walltime = "1:59:00"

# Keep the job executor just to keep it alive
job_executor, job_cluster = produce_flat_skim_from_track_skim.setup_job_framework(
    job_framework=job_framework,
    productions=productions,
    task_config=task_config,
    facility=facility,
    walltime=walltime,
    target_n_tasks_to_run_simultaneously=target_n_tasks_to_run_simultaneously,
    log_level=log_level,
    conda_environment_name=conda_environment_name,
)

[14:44:45] INFO     Requesting 1 core(s) in 8 block(s), with 1 tasks per block for 8 total     ]8;id=582138;file:///Users/REhlers/software/dev/mammoth/src/mammoth/helpers.py\mammoth.job_utils]8;;\:]8;id=697064;file:///Users/REhlers/software/dev/mammoth/src/mammoth/helpers.py#46\46]8;;\
                    tasks running simultaneously.                                                                  

## Status

In [6]:
job_executor

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:58409,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:58433,Total threads: 1
Dashboard: http://127.0.0.1:58440/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:58413,


## Submit jobs

In [7]:
futures = produce_flat_skim_from_track_skim.setup_and_submit_tasks(
    productions=productions,
    task_config=task_config,
    job_framework=job_framework,
    debug_mode=debug_mode,
    job_executor=job_executor if job_framework == job_utils.JobFramework.dask_delayed else None,  # type: ignore[arg-type]
)
#futures

[14:44:46] INFO     Tasks to execute:                     ]8;id=168979;file:///Users/REhlers/software/dev/mammoth/src/mammoth/hardest_kt/produce_flat_skim_from_track_skim.py\mammoth.hardest_kt.produce_flat_skim_from_track_skim]8;;\:]8;id=267973;file:///Users/REhlers/software/dev/mammoth/src/mammoth/hardest_kt/produce_flat_skim_from_track_skim.py#1157\1157]8;;\
                    ['calculate_data_skim']                                                                        

           INFO     Adding                                 ]8;id=757099;file:///Users/REhlers/software/dev/mammoth/src/mammoth/hardest_kt/produce_flat_skim_from_track_skim.py\mammoth.hardest_kt.produce_flat_skim_from_track_skim]8;;\:]8;id=930246;file:///Users/REhlers/software/dev/mammoth/src/mammoth/hardest_kt/produce_flat_skim_from_track_skim.py#441\441]8;;\
                    trains/pythia/2619/run_by_run/LHC18b8_                                                         
                    cent_woSDD/282008/1/AnalysisResults.18                                                         
                    b8_cent_woSDD.001.root for analysis                                                            

[14:44:47] INFO     Adding                                 ]8;id=442189;file:///Users/REhlers/software/dev/mammoth/src/mammoth/hardest_kt/produce_flat_skim_from_track_skim.py\mammoth.hardest_kt.produce_flat_skim_from_track_skim]8;;\:]8;id=405962;file:///Users/REhlers/software/dev/mammoth/src/mammoth/hardest_kt/produce_flat_skim_from_track_skim.py#441\441]8;;\
                    trains/pythia/2619/run_by_run/LHC18b8_                                                         
                    cent_woSDD/282031/2/AnalysisResults.18                                                         
                    b8_cent_woSDD.002.root for analysis                                                            

           INFO     Adding                                 ]8;id=735004;file:///Users/REhlers/software/dev/mammoth/src/mammoth/hardest_kt/produce_flat_skim_from_track_skim.py\mammoth.hardest_kt.produce_flat_skim_from_track_skim]8;;\:]8;id=121414;file:///Users/REhlers/software/dev/mammoth/src/mammoth/hardest_kt/produce_flat_skim_from_track_skim.py#441\441]8;;\
                    trains/pythia/2619/run_by_run/LHC18b8_                                                         
                    cent_woSDD/282120/3/AnalysisResults.18                                                         
                    b8_cent_woSDD.003.root for analysis                                                            

           INFO     Adding                                 ]8;id=94384;file:///Users/REhlers/software/dev/mammoth/src/mammoth/hardest_kt/produce_flat_skim_from_track_skim.py\mammoth.hardest_kt.produce_flat_skim_from_track_skim]8;;\:]8;id=178393;file:///Users/REhlers/software/dev/mammoth/src/mammoth/hardest_kt/produce_flat_skim_from_track_skim.py#441\441]8;;\
                    trains/pythia/2619/run_by_run/LHC18b8_                                                         
                    cent_woSDD/282146/4/AnalysisResults.18                                                         
                    b8_cent_woSDD.003.root for analysis                                                            

...

           INFO     Adding                                 ]8;id=295073;file:///Users/REhlers/software/dev/mammoth/src/mammoth/hardest_kt/produce_flat_skim_from_track_skim.py\mammoth.hardest_kt.produce_flat_skim_from_track_skim]8;;\:]8;id=286232;file:///Users/REhlers/software/dev/mammoth/src/mammoth/hardest_kt/produce_flat_skim_from_track_skim.py#441\441]8;;\
                    trains/pythia/2619/run_by_run/LHC18b8_                                                         
                    fast/282146/18/AnalysisResults.18b8_fa                                                         
                    st.007.root for analysis                                                                       

           INFO     Adding                                 ]8;id=5080;file:///Users/REhlers/software/dev/mammoth/src/mammoth/hardest_kt/produce_flat_skim_from_track_skim.py\mammoth.hardest_kt.produce_flat_skim_from_track_skim]8;;\:]8;id=189680;file:///Users/REhlers/software/dev/mammoth/src/mammoth/hardest_kt/produce_flat_skim_from_track_skim.py#441\441]8;;\
                    trains/pythia/2619/run_by_run/LHC18b8_                                                         
                    fast/282123/19/AnalysisResults.18b8_fa                                                         
                    st.012.root for analysis                                                                       

           INFO     Adding                                 ]8;id=908619;file:///Users/REhlers/software/dev/mammoth/src/mammoth/hardest_kt/produce_flat_skim_from_track_skim.py\mammoth.hardest_kt.produce_flat_skim_from_track_skim]8;;\:]8;id=299278;file:///Users/REhlers/software/dev/mammoth/src/mammoth/hardest_kt/produce_flat_skim_from_track_skim.py#441\441]8;;\
                    trains/pythia/2619/run_by_run/LHC18b8_                                                         
                    fast/282031/20/AnalysisResults.18b8_fa                                                         
                    st.005.root for analysis                                                                       

           INFO     Accumulated 10324 futures for pythia  ]8;id=472991;file:///Users/REhlers/software/dev/mammoth/src/mammoth/hardest_kt/produce_flat_skim_from_track_skim.py\mammoth.hardest_kt.produce_flat_skim_from_track_skim]8;;\:]8;id=513241;file:///Users/REhlers/software/dev/mammoth/src/mammoth/hardest_kt/produce_flat_skim_from_track_skim.py#1195\1195]8;;\

           INFO     Accumulated 10324 total futures       ]8;id=620726;file:///Users/REhlers/software/dev/mammoth/src/mammoth/hardest_kt/produce_flat_skim_from_track_skim.py\mammoth.hardest_kt.produce_flat_skim_from_track_skim]8;;\:]8;id=455836;file:///Users/REhlers/software/dev/mammoth/src/mammoth/hardest_kt/produce_flat_skim_from_track_skim.py#1197\1197]8;;\

In [8]:
futures[:20]

[
    <Future: pending, key: _run_data_skim-a1d8df5e-4561-46c8-81bd-901733096c29>,
    <Future: pending, key: _run_data_skim-fc47ad3d-e7a7-4d09-be7d-30975e014f2b>,
    <Future: pending, key: _run_data_skim-70b46a87-c410-415d-a84e-5994a9560e40>,
    <Future: pending, key: _run_data_skim-bcd7cd61-1277-4389-bc3a-9bec70a9c7e7>,
    <Future: pending, key: _run_data_skim-40d9f5c5-83d2-4de5-b5ac-050554ca3670>,
    <Future: pending, key: _run_data_skim-9d0c1c09-b16b-4e60-99e9-b6a84709435e>,
    <Future: pending, key: _run_data_skim-5cde0770-f7bb-4391-9b1d-81d554ff11bd>,
    <Future: pending, key: _run_data_skim-42177e36-fc38-494d-81b5-85e65a2ebe13>,
    <Future: pending, key: _run_data_skim-a2e3156b-9a89-4c11-8c45-cfd19849e8a3>,
    <Future: pending, key: _run_data_skim-ba911b10-fde2-42cb-b420-19c1ea8e7d9a>,
    <Future: pending, key: _run_data_skim-34ae705f-0376-4f19-8540-7fec212d2f7c>,
    <Future: pending, key: _run_data_skim-c98fdb94-9f97-4d31-8304-e31c73a470ea>,
    <Future: pending, key: _run_data_skim-4fa8ce40-f6ad-4fcd-baa0-b98f4c0e23c2>,
    <Future: pending, key: _run_data_skim-3ffeff02-4606-4c3c-9365-3477c2e4a118>,
    <Future: pending, key: _run_data_skim-c3e5d52d-4f43-48d1-873e-29bc5d2fa79a>,
    <Future: pending, key: _run_data_skim-96db8405-5ef9-4956-84df-1fa27719d627>,
    <Future: pending, key: _run_data_skim-06621fb2-93da-4165-87de-55cab983e0ea>,
    <Future: pending, key: _run_data_skim-d193e89d-71de-4386-a4fe-355ee68705da>,
    <Future: pending, key: _run_data_skim-489458fe-bb52-416f-b958-d47df003317d>,
    <Future: pending, key: _run_data_skim-ae3990a0-e764-46fc-9fb9-93cdac516358>
]

In [ ]:
all((f.result() for f in futures[2:]))

## Cleanup

In [8]:
job_executor.close()
job_cluster.close()

2023-02-13 14:31:16,378 - distributed.nanny - WARNING - Worker process still alive after 3.199999389648438 seconds, killing


[14:31:16] WARNING  Worker process still alive after 3.199999389648438 seconds, killing       ]8;id=21804;file:///Users/REhlers/software/dev/mammoth/.venv/lib/python3.10/site-packages/distributed/nanny.py\distributed.nanny]8;;\:]8;id=205644;file:///Users/REhlers/software/dev/mammoth/.venv/lib/python3.10/site-packages/distributed/nanny.py#831\831]8;;\

2023-02-13 14:31:16,383 - distributed.nanny - WARNING - Worker process still alive after 3.199999389648438 seconds, killing


           WARNING  Worker process still alive after 3.199999389648438 seconds, killing       ]8;id=578091;file:///Users/REhlers/software/dev/mammoth/.venv/lib/python3.10/site-packages/distributed/nanny.py\distributed.nanny]8;;\:]8;id=796985;file:///Users/REhlers/software/dev/mammoth/.venv/lib/python3.10/site-packages/distributed/nanny.py#831\831]8;;\

2023-02-13 14:31:16,389 - distributed.nanny - WARNING - Worker process still alive after 3.199999389648438 seconds, killing


...

           WARNING  Worker process still alive after 3.1999992370605472 seconds, killing      ]8;id=775482;file:///Users/REhlers/software/dev/mammoth/.venv/lib/python3.10/site-packages/distributed/nanny.py\distributed.nanny]8;;\:]8;id=852355;file:///Users/REhlers/software/dev/mammoth/.venv/lib/python3.10/site-packages/distributed/nanny.py#831\831]8;;\

2023-02-13 14:31:16,419 - distributed.nanny - WARNING - Worker process still alive after 3.2 seconds, killing


           WARNING  Worker process still alive after 3.2 seconds, killing                     ]8;id=988334;file:///Users/REhlers/software/dev/mammoth/.venv/lib/python3.10/site-packages/distributed/nanny.py\distributed.nanny]8;;\:]8;id=129160;file:///Users/REhlers/software/dev/mammoth/.venv/lib/python3.10/site-packages/distributed/nanny.py#831\831]8;;\

2023-02-13 14:31:16,421 - distributed.nanny - WARNING - Worker process still alive after 3.2 seconds, killing


           WARNING  Worker process still alive after 3.2 seconds, killing                     ]8;id=467480;file:///Users/REhlers/software/dev/mammoth/.venv/lib/python3.10/site-packages/distributed/nanny.py\distributed.nanny]8;;\:]8;id=655754;file:///Users/REhlers/software/dev/mammoth/.venv/lib/python3.10/site-packages/distributed/nanny.py#831\831]8;;\